# library

In [1]:
# 사용할 패키지를 import합니다
import sys
print('python', sys.version)

import numpy as np
print('numpy', np.__version__)

import pandas as pd
print('pandas', pd.__version__)

import matplotlib as mpl
print('matplotlib', mpl.__version__)

from tqdm import tqdm

import matplotlib.pyplot as plt
plt.rc("font", family="Malgun Gothic") # window
plt.rc("axes", unicode_minus=False) # x,y축 (-)부호 표시

import seaborn as sns
print('pandas', sns.__version__)

#from venn import venn
import plotly.express as px

from sklearn.preprocessing import StandardScaler
#import GEOparse ## pip install GEOparse

# 결과 확인을 용이하게 하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

python 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
numpy 1.20.3
pandas 1.3.4
matplotlib 3.4.3
pandas 0.11.2


In [2]:
import pickle

In [3]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.decomposition import PCA
#import prince

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix

# modeling

## ori data

In [5]:
with open("mer_exp_qtNorm.pickle","rb") as fr:
    mer_exp_qtNorm = pickle.load(fr)

In [6]:
a = list(mer_exp_qtNorm.columns)
df = pd.DataFrame(a)

lst_targets = []
for i in range(189):
    lst_targets.append('met')
for j in range(266):
    lst_targets.append('ctc')
for k in range(28):
    lst_targets.append('prm')
    
int_targets = []
for i in lst_targets:
    if i == 'met':
        int_targets.append(0)
    elif i == 'ctc':
        int_targets.append(1)
    elif i == 'prm':
        int_targets.append(2)

    
targets = pd.DataFrame([lst_targets, int_targets]).T

target_df = pd.concat([df,targets], axis = 1)
target_df.columns = ['name', 'target', 'target_int']
target_df = target_df.set_index('name')


mer_exp_qtNorm1 = mer_exp_qtNorm
mer_exp_qtNorm1 = mer_exp_qtNorm1.T
mer_exp_qtNorm1 = pd.merge(mer_exp_qtNorm1, target_df, how = 'left', left_on = mer_exp_qtNorm1.index, right_on = target_df.index)

mer_exp_qtNorm1 = mer_exp_qtNorm1.astype({'target_int' : 'int'})
mer_exp_qtNorm1

,key_0,A1CF,A2M,A4GALT,AAAS,AADAC,AAGAB,AAK1,AANAT,AARSD1,...,ZSCAN18,ZSCAN2,ZWILCH,ZWINT,ZXDB,ZXDC,ZZEF1,ZZZ3,target,target_int
0,GSM352095(Lung),-0.762161,1.180796,-0.137791,0.599584,-0.729800,0.182417,-0.258430,-0.801284,0.045837,...,0.564190,-0.414918,0.440365,1.186005,-0.494892,-0.196995,0.046822,0.812887,met,0
1,GSM352097(Brain),0.815680,1.160821,-0.342571,1.111625,-0.729800,-0.058388,-0.273952,-0.835241,0.385127,...,1.099553,-0.167976,0.892148,3.745774,-0.654508,-0.337004,-0.062373,0.873747,met,0
2,GSM352098(Brain),-0.748810,0.967269,-0.308824,0.813322,-0.719202,0.492152,-0.195600,-0.826537,0.056665,...,0.838842,-0.101950,0.879677,3.138333,-0.615568,0.148753,-0.104008,0.898081,met,0
3,GSM352100(Bone),-0.772205,1.308681,-0.085146,0.622287,-0.750873,0.259798,-0.310480,-0.816698,0.026232,...,0.700339,-0.296512,0.760778,2.719735,-0.642044,-0.154915,-0.074523,1.083389,met,0
4,GSM352101(Brain),-0.741059,0.861349,-0.346662,0.810402,-0.694349,0.576679,0.022478,-0.810466,1.310174,...,0.554929,-0.292044,0.719036,3.605673,-0.609100,0.147914,-0.060281,1.235690,met,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,ACH-000352(pleural_effusion),-0.647676,-0.559259,1.453718,1.014213,-0.694501,1.414127,-0.169054,-0.430839,1.043400,...,0.848161,-0.223030,0.460952,2.911773,-0.344089,0.265916,0.303025,0.358825,prm,2
479,ACH-000759(pleural_effusion),-0.695093,0.797722,-0.408008,0.871683,-0.423008,1.316980,0.159681,-0.695093,0.954912,...,0.781944,-0.055693,0.033666,2.350092,-0.268697,0.624050,0.091298,-0.234729,prm,2
480,ACH-000017(pleural_effusion),-0.731862,-0.556405,-0.556405,1.293704,-0.550165,0.816187,-0.075605,-0.492283,0.542203,...,-0.534146,-0.261349,0.558538,3.443355,-0.292708,0.145318,0.181554,0.329267,prm,2
481,ACH-000573(pleural_effusion),-0.768729,-0.541974,0.380335,0.785142,-0.598003,1.288936,0.277126,-0.608924,1.499700,...,0.663797,-0.312120,0.828520,2.167716,-0.315726,-0.138110,0.144447,0.660052,prm,2


In [7]:
with open("mer_exp_qtNorm1.pickle","wb") as fw:
    pickle.dump(mer_exp_qtNorm1, fw)

In [8]:
X = mer_exp_qtNorm1.iloc[:,1:-2]
y = mer_exp_qtNorm1.loc[:, ['target']]

In [9]:
mer_exp_qtNorm1_xg = mer_exp_qtNorm1.drop(['target'], axis = 1)
Xx = mer_exp_qtNorm1_xg.iloc[:, 1:-1]
Yx = mer_exp_qtNorm1_xg.loc[:, ['target_int']]

### random forest

In [10]:
result_dic = {}
for i in tqdm(range(30)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    rn = RandomForestClassifier(oob_score=True)
    model = rn.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)

    result_dic["iter_{}".format(i+1)]= ["RF",model.score(X_test, y_test), metrics.precision_score(y_test, y_pred, average = 'macro'),
                 metrics.recall_score(y_test, y_pred, average = 'macro'), metrics.f1_score(y_test, y_pred, average = 'macro')]
    
df_RF = pd.DataFrame(result_dic).T
df_RF.columns = ['model',"accuracy","precision","recall","f1"]

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2314243372.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(X_train, y_train)
  3%|██▊                                                                                | 1/30 [00:03<01:38,  3.41s/it]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2314243372.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(X_train, y_train)
  7%|█████▌                                                                             | 2/30 [00:06<01:33,  3.34s/it]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2314243372.py:5: DataConversionWarning: A column-vector y was passed when a 1d 

 67%|██████████████████████████████████████████████████████▋                           | 20/30 [01:03<00:30,  3.07s/it]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2314243372.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(X_train, y_train)
 70%|█████████████████████████████████████████████████████████▍                        | 21/30 [01:07<00:27,  3.09s/it]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2314243372.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(X_train, y_train)
 73%|████████████████████████████████████████████████████████████▏                     | 22/30 [01:10<00:24,  3.05s/it]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2314243372.py:5: DataConversionWarning: A column-vector y was passed when a 1d 

### xgbboost

In [13]:
result_dic = {}
for i in tqdm(range(30)):
    Xx_train, Xx_test, yx_train, yx_test = train_test_split(Xx, Yx, test_size=0.2)
    xgb = XGBClassifier(n_estimators=100, learning_rate=0.01, max_depth=4)
    model = xgb.fit(Xx_train, yx_train)
    
    y_pred = model.predict(Xx_test)
    y_pred_prob = model.predict_proba(Xx_test)
    
    result_dic["iter_{}".format(i+1)]= ["Xgb",model.score(Xx_test, yx_test), metrics.precision_score(yx_test, y_pred, average = 'macro'),
                 metrics.recall_score(yx_test, y_pred, average = 'macro'), metrics.f1_score(yx_test, y_pred, average = 'macro')]

    
df_xgb = pd.DataFrame(result_dic).T
df_xgb.columns = ['model',"accuracy","precision","recall","f1"]

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [10:55<00:00, 21.86s/it]


### adaboost

In [14]:
result_dic = {}
for i in tqdm(range(30)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
    model = abc.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)
    
    result_dic["iter_{}".format(i+1)]= ["Ada",model.score(X_test, y_test), metrics.precision_score(y_test, y_pred, average = 'macro'),
                 metrics.recall_score(y_test, y_pred, average = 'macro'), metrics.f1_score(y_test, y_pred, average = 'macro')]
    
df_ada = pd.DataFrame(result_dic).T
df_ada.columns = ['model',"accuracy","precision","recall","f1"]

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  3%|██▊                                                                                | 1/30 [00:27<13:12, 27.34s/it]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  7%|█████▌                                                                             | 2/30 [00:55<13:06, 28.08s/it]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-ve

 63%|███████████████████████████████████████████████████▉                              | 19/30 [09:12<05:18, 29.00s/it]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 67%|██████████████████████████████████████████████████████▋                           | 20/30 [09:39<04:45, 28.57s/it]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 70%|█████████████████████████████████████████████████████████▍                        | 21/30 [10:08<04:17, 28.60s/it]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-ve

## pca data

In [15]:
with open("mer_exp_pca.pickle","rb") as fr:
    mer_exp_pca = pickle.load(fr)

In [16]:
a = list(mer_exp_pca.index)
df = pd.DataFrame(a)

lst_targets = []
for i in range(189):
    lst_targets.append('met')
for j in range(266):
    lst_targets.append('ctc')
for k in range(28):
    lst_targets.append('prm')
    
int_targets = []
for i in lst_targets:
    if i == 'met':
        int_targets.append(0)
    elif i == 'ctc':
        int_targets.append(1)
    elif i == 'prm':
        int_targets.append(2)

    
targets = pd.DataFrame([lst_targets, int_targets]).T
target_df = pd.concat([df,targets], axis = 1)
target_df.columns = ['name', 'target', 'target_int']
target_df = target_df.set_index('name')


mer_exp_pca1 = mer_exp_pca

mer_exp_pca1 = pd.merge(mer_exp_pca1, target_df, how = 'left', left_on = mer_exp_pca1.index, right_on = target_df.index)
mer_exp_pca1 = mer_exp_pca1.astype({'target_int' : 'int'})
mer_exp_pca1

,key_0,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,...,pca_52,pca_53,pca_54,pca_55,pca_56,pca_57,pca_58,pca_59,target,target_int
0,GSM352095(Lung),-28.150528,3.253752,38.453725,0.588526,-0.390641,0.110821,6.599275,-20.633990,14.345296,...,1.993956,-2.259019,2.010595,1.002164,2.895003,1.532487,-2.506051,2.667427,met,0
1,GSM352097(Brain),-26.842402,4.139521,42.863831,0.947708,0.989828,-4.810706,5.298568,-20.459387,16.234951,...,-0.532824,-4.322230,-1.476458,-0.116561,1.617759,-0.487162,-1.096093,-2.191144,met,0
2,GSM352098(Brain),-29.712101,-0.027494,37.028732,0.210543,5.952633,3.138591,2.957188,-24.103836,13.916650,...,-1.201567,0.484970,-2.957265,-1.684462,-0.017679,-0.965762,-1.078973,-2.591676,met,0
3,GSM352100(Bone),-26.358055,8.144138,47.869513,0.489725,3.806026,8.301494,7.317130,-19.456005,20.485817,...,-0.326943,1.943351,1.287358,0.745712,-1.220863,1.214800,-0.238797,0.146541,met,0
4,GSM352101(Brain),-28.062230,-2.372655,35.565397,-1.525383,5.612493,-0.102849,1.796683,-24.880327,16.367740,...,-1.820117,0.263627,0.266745,-0.057359,-0.130358,-0.891864,-0.135826,-1.480134,met,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,ACH-000352(pleural_effusion),-23.814368,-22.951112,-2.203806,-9.274524,-10.780583,-23.578187,4.335292,0.481987,-3.249898,...,-0.202530,2.681531,0.705252,-0.322326,-1.669274,1.647648,-1.347515,3.438507,prm,2
479,ACH-000759(pleural_effusion),-23.878335,-23.437875,-1.150795,-7.025613,-8.489253,-16.286388,-0.290822,3.156788,-3.074727,...,-1.006203,0.377389,2.281701,-1.235227,-1.570138,1.089647,-2.594377,-1.415521,prm,2
480,ACH-000017(pleural_effusion),-25.758045,-24.980724,-2.781498,-6.880136,-9.685853,-20.169124,4.804096,2.355502,-5.754801,...,0.403325,1.183905,1.434220,-0.402552,-1.631396,1.281267,-1.156837,0.026475,prm,2
481,ACH-000573(pleural_effusion),-21.520941,-18.427801,4.953522,-10.335315,-14.250924,-25.132353,18.356065,12.392616,-2.264742,...,0.657179,-1.855411,-0.517298,2.526051,2.827696,-1.970025,1.913963,-1.416848,prm,2


In [17]:
pX = mer_exp_pca1.iloc[:,1:-2]
py = mer_exp_pca1.loc[:, ['target']]

mer_exp_pca1_pxg = mer_exp_pca1.drop(['target'], axis = 1)
pXx = mer_exp_pca1_pxg.iloc[:,1:-1]
pyx = mer_exp_pca1_pxg.loc[:, ['target_int']]

### randomforest

In [18]:
result_dic = {}
for i in tqdm(range(30)):
    pX_train, pX_test, py_train, py_test = train_test_split(pX, py, test_size=0.2)
    rn = RandomForestClassifier(oob_score=True)
    model = rn.fit(pX_train, py_train)

    y_pred = model.predict(pX_test)
    y_pred_prob = model.predict_proba(pX_test)

    result_dic["iter_{}".format(i+1)]= ["RF",model.score(pX_test, py_test), metrics.precision_score(py_test, y_pred, average = 'macro'),
                 metrics.recall_score(py_test, y_pred, average = 'macro'), metrics.f1_score(py_test, y_pred, average = 'macro')]
    
df_P_RF = pd.DataFrame(result_dic).T
df_P_RF.columns = ['model',"accuracy","precision","recall","f1"]

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/3809788636.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(pX_train, py_train)
  3%|██▊                                                                                | 1/30 [00:00<00:18,  1.59it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/3809788636.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(pX_train, py_train)
  7%|█████▌                                                                             | 2/30 [00:01<00:17,  1.64it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/3809788636.py:5: DataConversionWarning: A column-vector y was passed when a

 73%|████████████████████████████████████████████████████████████▏                     | 22/30 [00:14<00:05,  1.57it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/3809788636.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(pX_train, py_train)
 77%|██████████████████████████████████████████████████████████████▊                   | 23/30 [00:14<00:04,  1.58it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/3809788636.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = rn.fit(pX_train, py_train)
 80%|█████████████████████████████████████████████████████████████████▌                | 24/30 [00:15<00:03,  1.62it/s]C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/3809788636.py:5: DataConversionWarning: A column-vector y was passed when a

### xgbboost

In [19]:
result_dic = {}
for i in tqdm(range(30)):
    pXx_train, pXx_test, pyx_train, pyx_test = train_test_split(pXx, pyx, test_size=0.2) 
    xgb = XGBClassifier(n_estimators=100, learning_rate=0.01, max_depth=4)
    model = xgb.fit(pXx_train, pyx_train)
    
    y_pred = model.predict(pXx_test)
    y_pred_prob = model.predict_proba(pXx_test)
    
    result_dic["iter_{}".format(i+1)]= ["Xgb",model.score(pXx_test, pyx_test), metrics.precision_score(pyx_test, y_pred, average = 'macro'),
                 metrics.recall_score(pyx_test, y_pred, average = 'macro'), metrics.f1_score(pyx_test, y_pred, average = 'macro')]

    
df_P_xgb = pd.DataFrame(result_dic).T
df_P_xgb.columns =['model',"accuracy","precision","recall","f1"]

 63%|███████████████████████████████████████████████████▉                              | 19/30 [00:12<00:07,  1.54it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:19<00:00,  1.53it/s]


### adaboost

In [20]:
result_dic = {}
for i in tqdm(range(30)):
    pX_train, pX_test, py_train, py_test = train_test_split(pX, py, test_size=0.2)
    abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
    model = abc.fit(pX_train, py_train)

    y_pred = model.predict(pX_test)
    y_pred_prob = model.predict_proba(pX_test)
    
    result_dic["iter_{}".format(i+1)]= ["Ada",model.score(pX_test, py_test), metrics.precision_score(py_test, y_pred, average = 'macro'),
                 metrics.recall_score(py_test, y_pred, average = 'macro'), metrics.f1_score(py_test, y_pred, average = 'macro')]
    
df_P_ada = pd.DataFrame(result_dic).T
df_P_ada.columns = ['model',"accuracy","precision","recall","f1"]

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  3%|██▊                                                                                | 1/30 [00:00<00:14,  2.06it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
  7%|█████▌                                                                             | 2/30 [00:01<00:14,  1.97it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-ve

 63%|███████████████████████████████████████████████████▉                              | 19/30 [00:09<00:05,  2.12it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 67%|██████████████████████████████████████████████████████▋                           | 20/30 [00:09<00:04,  2.13it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
 70%|█████████████████████████████████████████████████████████▍                        | 21/30 [00:10<00:04,  2.14it/s]C:\Users\qpflw\anaconda3\lib\site-packages\sklearn\utils\validation.py:1141: DataConversionWarning: A column-ve

# oir data 모델 성능merge

In [21]:
all_df = pd.concat([df_RF, df_xgb, df_ada])
all_pca_df = pd.concat([df_P_RF, df_P_xgb, df_P_ada])
model_df = pd.concat([all_df, all_pca_df])

In [22]:
name_lst = []
for i in range(90):
    name_lst.append('ori')
for i in range(90):
    name_lst.append('pca')
model_df['type'] = name_lst
model_df

,model,accuracy,precision,recall,f1,type
iter_1,RF,0.845361,0.901795,0.645547,0.659167,ori
iter_2,RF,0.783505,0.608875,0.608995,0.608183,ori
iter_3,RF,0.865979,0.753788,0.663035,0.679924,ori
iter_4,RF,0.886598,0.770161,0.674023,0.693735,ori
iter_5,RF,0.845361,0.791608,0.788053,0.789744,ori
...,...,...,...,...,...,...
iter_26,Ada,0.814433,0.665813,0.620174,0.633359,pca
iter_27,Ada,0.536082,0.377283,0.414279,0.353571,pca
iter_28,Ada,0.731959,0.493056,0.516922,0.497604,pca
iter_29,Ada,0.474227,0.340563,0.398574,0.31752,pca


In [23]:
with open("model_df.pickle","wb") as fw:
    pickle.dump(model_df, fw)

# feature_importance_XGB

In [24]:
feature_importance_XGB = pd.DataFrame()
for i in tqdm(range(30)):
    Xx_train, Xx_test, yx_train, yx_test = train_test_split(Xx, Yx, test_size=0.2)
    xgb = XGBClassifier(n_estimators=100, learning_rate=0.01, max_depth=4)
    model = xgb.fit(Xx_train, yx_train)
    
    feature_importance_XGB['feature'] = mer_exp_qtNorm1.columns[1:-2]
    feature_importance_XGB['iter_{}'.format(i+1)] = model.feature_importances_
    

feature_importance_XGB['mean'] = feature_importance_XGB.mean(axis=1)
feature_importance_XGB = feature_importance_XGB.sort_values('mean', ascending = False)
feature_importance_XGB = feature_importance_XGB.T
feature_importance_XGB.columns = feature_importance_XGB.loc['feature']
feature_importance_XGB = feature_importance_XGB.drop(['feature'])

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [10:15<00:00, 20.51s/it]
C:\Users\qpflw\AppData\Local\Temp/ipykernel_24596/2005821347.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  feature_importance_XGB['mean'] = feature_importance_XGB.mean(axis=1)


In [25]:
with open("feature_importance_XGB.pickle","wb") as fw:
    pickle.dump(feature_importance_XGB, fw)